In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast
import time
today = time.strftime("%Y_%m_%d")
from enrich_dimensions.rounds import rounds, rounding_inch_feet,rounding_lbs,rounding_w,rounding_oz, rounding_lb,rounding_gal, re_extract, curate, round_string_float, clean_data,reg_ex,rounding_period_after_unit 
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

def get_unique_list(material_list):
    unique_list = []
    for item in material_list:
        if isinstance(item, str) and "[" in item and "]" in item:
            item_values = ast.literal_eval(item)
            unique_list.extend(item_values)
        else:
            unique_list.append(item)
    return sorted(list(set(unique_list)))

customer_id = '90'
customer_name='%wbmason%'
client='WBMason'

In [2]:
attribut = 'quantity'
dateszs='2001-08-11'
dateszs='2001-11-01'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs,
         'attribut':attribut}

print('Start Query of Curated Data')
cb = query_from_file(file_name='query/curated_all_attributes_date_family.sql', params=params)
print(len(cb))

fill_dict = cb.groupby('external_id')['buckets'].last().to_dict()
cb['buckets'] = cb['buckets'].fillna(cb['external_id'].map(fill_dict))

Start Query of Curated Data
767429


# Check attribute bucket pairs 

In [107]:
print(len(sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())))
lst_attribute=sorted(cb['attribute'].explode().value_counts().reset_index()['index'].to_list())
print('Number of attributes: '+str(len(lst_attribute)))
print('')


# a='height'
# lst_attribute=[a]


for i in range(len(lst_attribute)):
    attri=lst_attribute[i]
    dateszs='2001-08-11'
    attribut=attri
    dateszs='2001-11-01'
    curation_col = f'Q:{attribut}'
    params = {'customer_id': customer_id,
              'customer_name':customer_name,
             'dateszs':dateszs,
             'attribut':attribut}

    print('Start Buckets')
    print(str(i)+' Attribute: '+str(attri))
    bucket_value = query_from_file(file_name='query/Bucket_Value_Strategy.sql', params=params)

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
    print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    brand=cb[(cb['value'].astype(str)!='["n/a"]')&(cb['value'].astype(str)!='n/a')&(cb['attribute'].astype(str)==attri)]
#     print('Number of SKUs for the attribute '+str(attri)+': '+str(len(brand)))
    list_of_all_buckets=sorted(list(set(bucket_value['buckets'].to_list())))

    filtered_dfs = {}
    b_lst=[]

    for i in range(len(list_of_all_buckets)):

        # values that are supposed to be in the buckets
        action_bucket_values=bucket_value[bucket_value['buckets'].astype(str)==list_of_all_buckets[i]]['values'].to_list()

        # values that are actually in the curation
        action=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]['value'].to_list()

        unique_values=get_unique_list(action)

        target=[x for x in unique_values if x not in action_bucket_values]
        if len(target) > 0:
            target = target[0]
            pat=f'''(?i)({target})|()'''
            bucket_filtered_df=brand[brand['buckets'].astype(str)==list_of_all_buckets[i]]
            bucket_filtered_df['match']=bucket_filtered_df['value'].apply(lambda x: re_extract(pat,str(x)))
            fil=bucket_filtered_df[bucket_filtered_df['match'].astype(str)!='[]']
            filtered_dfs[list_of_all_buckets[i]] = fil
            b_lst.append(list_of_all_buckets[i])
        else:
            pass

    
    b_lst[0:10]
    valuez = {}
    external_id_list=[]
    for i in range(len(b_lst)):
        valuez[b_lst[i]]=filtered_dfs[b_lst[i]][['external_id','value']]
        external_id_list.append(filtered_dfs[b_lst[i]]['external_id'].tolist())
    import itertools
    flattened_list = list(itertools.chain.from_iterable(external_id_list))
    if len(flattened_list)>0:
        print('Buckets Effected: '+str(len(b_lst)))
        df_name = f'match_{attri}'
        data = {'external_id': flattened_list, f'Q:{attri}': '[]'}
        df_dict = {df_name: pd.DataFrame(data)}
        new_df = df_dict[df_name]
        print('Number of SKUs to be wiped: '+str(len(new_df)))
        print('')
        print('')

        def get_df_name(df):
            name =[x for x in globals() if globals()[x] is df][0]
            return name
        def looks_good(customer, matches,attri): 
            drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
            matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attri}-{today}.csv',index=False) 
        looks_good(client, new_df,attri) 
    else:
        pass

398
Number of attributes: 398

Start Buckets
0 Attribute: count
Number of SKUs for the attribute count: 47695
Buckets Effected: 5
Number of SKUs to be wiped: 941




In [106]:
x=brand[brand['external_id'].isin(flattened_list)]
x['value'].explode().value_counts()

2-12 Count       765
250-500 Count     89
19 - 24 Count     87
Name: value, dtype: int64

In [20]:
x[f'Q:{a}']=x['value'].apply(lambda x: re.sub(r'(?<=oz)\.','',str(x)))

In [28]:
x[f'Q:{a}'].explode().value_counts()

In [22]:
match_cap=x[['external_id',f'Q:{a}']]

# Check freetext attribute

In [102]:
amp=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+)?(?!\,)A)|(n\/a)|()'''
btu=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+|\,\d+)?(?!\,) BTU)|(n\/a)|()'''
can=r'''((?<!\d)(?<!\,)\d+\-can)|(n\/a)|()'''
cubic_feet=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+)?(?!\,) cu ft)|(n\/a)|()'''
cup=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+)?(?!\,)\-Cup)|(n\/a)|()'''
db=r'((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+)?(?!\,) (?:dB))|(n\/a)|()'
dots_per_inch_number_X=r'''((?<!\,)(?<!\d)(?<!X )\d+(?!\,)(?: x \d+)? DPI)|(n\/a)|()'''
dynamic_load_capacity=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+)?(?!\,) (?:lb))|(n\/a)|()'''

feet_thousand_seperator=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+(?:\.\d+)?|\.\d+)?(?!\,) (?:ft|in))|(n\/a)|()'''
feet_mm=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+(?:\.\d+)?|\.\d+)?(?!\,) (?:m|in))|(n\/a)|()'''
feet_mi=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,) (?:ft|mi))|(n\/a)|()'''
flow_rate_GPM=r'''(\d+ GPM)|(n\/a)|()'''
flow_rate_GPM=r'''(\d+ GPM)|(n\/a)|()'''
frequency_rate_hz=r'''((?<!\,)(?<!\d)\d+(?:\,\d+)? (?:Hz))|(n\/a)|()'''
g_force=r'''(\d+G)|(n\/a)|()'''
gauge=r'''(\d+ Gauge)|(n\/a)|()'''
gb_tb_mb=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+)?(?!\,)(?:TB|GB|MB))|(n\/a)|()'''
ghz=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,) (?:GHz))|(n\/a)|()'''


horsepower=r'(\d+ HP)|(n\/a)|()'
hour=r'(\d+(?:\.\d+)? hr)|(n\/a)|()'
hours=r'((?<!\d)(?<!\,)\d+(?:\,\d+)?(?!\,)(?!\d) hrs)|(n\/a)|()'
hz=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,) (?:Hz))|(n\/a)|()'''
inch=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+)?(?!\,) in)|(n\/a)|()'''
inch_feet=r'((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+|\,\d+)?(?!\,) (?:in|ft|m))|(n\/a)|()'
inch_fraction=r'''(\d+(?:(?:\.\d+)?|\d+\-\d+\/\d+|(?: |\-)\d+\/\d+|\/\d+) in)|(n\/a)|()'''
inch_mm=r'''((?<!\d)(?<!\,)\d+(?:\-\d+\/\d+|\/\d+)?(?!\,)(?!\d) (?:in|mm))|(n\/a)|()'''


mega_pixels=r'''(\d+ MP)|(n\/a)|()'''
mg=r'(\d+ mg)|(n\/a)|()'
micron=r'''(\d+ micron)|(n\/a)|()'''
mil=r'''(\d+(?:\.\d+)? mil)|(n\/a)|()'''
milliamp_hours=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+)?(?!\,)(?:mAh))|(n\/a)|()'''
minute=r'''((?<!\d)(?<!\,)\d+ min)|(n\/a)|()'''
mm=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+)?(?!\,) mm)|(n\/a)|()'''
ms=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,) (?:ms))|(n\/a)|()'''
nit=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+)?(?!\,) nit)|(n\/a)|()'''
number_space_fraction=r'''(\d+(?:(?:\.\d+)?|\d+\-\d+\/\d+| \d+\/\d+) in)|(n\/a)|()'''
number_with_sperator=r'''((?<!\d)\d+(?:\,\d+|\d\d)?(?!\,)(?!\d))|(n\/a)|()'''
number_no_seperator=r'''((?<!\d)(?<!\,)\d+(?!\,)(?!\d))|(n\/a)|()'''


page=r'''(\d+ pages)|(n\/a)|()'''
peak_transfer_speed=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,) (?:Gbps|Mbps|Kbps|MBps|GBps))|(n\/a)|()'''
person=r'''(1 person)|([2-9] people)|(\d\d+ people)|(n\/a)|()'''
pieces=r'''(1 Piece)|((?:\d\d+|[2-9]) Pieces)|(n\/a)|()'''
pound=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\.\d+|\/\d+|\,\d+)?(?!\,) lb)|(n\/a)|()'''
psi=r'(\d+ PSI)|(n\/a)|()'


ram=r'''((?<!\,)(?<!\d)(?<!X )\d+(?:\,\d+|\.\d+)?(?!\,)(?:GB|MB))|(n\/a)|()'''
rpm=r'((?<!\d)(?<!\,)\d+(?:\,\d+)?(?!\,)(?!\d) RPM)|(n\/a)|()'
sec=r'(\d+(?:\.\d+)? sec)|(n\/a)|()'
square_feet=r'(\d+(?:\,\d+)? sq ft)|(n\/a)|()'
tpi=r'''(\d+ tpi)|(n\/a)|()'''
temperature=r'''(\d+ F)|(n\/a)|()'''
volt=r'''(\d+V)|(n\/a)|()'''
wat=r'''(\d+W)|(n\/a)|()'''
year=r'((?:[2-9]|\d\d+) years)|(1 year)|(\d+ months)|(n\/a)|()'
# awg=r'''(\d+AWG)|(n\/a)|()'''









patterns = {
    'additional_range': square_feet,
    'amperage': amp,
    'appliance_cubic_feet': cubic_feet,
    'arbor_hole': inch,
    'bills_per_minute': number_with_sperator,
    'bin_count': number_no_seperator,
    'blade_length':inch_fraction,
    'blade_width': inch_fraction,
    'bond_lbs':pound,
    'bond_time':sec,
    'brew_capacity':cup,
    'brightness_nit':nit,
    'btu':btu,
    'can_count':can,
    'channel_count':number_no_seperator,
    'characters_per_line':number_no_seperator,
    'charging_time_minutes':minute,
    'cleaning_path':number_space_fraction,
    'clip_capacity':inch,
    'coins_per_minute':number_no_seperator,
    'cord_length':inch_feet,
    'core_diameter':inch,
    'cut_length':inch_fraction,
    'daily_ice_production':pound,
    'decibels_d_b':db,
    'depth':inch_feet,
    'dial_width':inch,
    'diameter':inch,
    'diameter_metric':mm,
    'diameter_sae':inch,
    'dots_per_inch_dpi':number_with_sperator,
    'dots_per_inch_mice_dpi':number_with_sperator,
    'dots_per_inch_printers_scanners_dpi':dots_per_inch_number_X,
    'drive_capacity':gb_tb_mb,
    'drive_size':inch,
    'dynamic_load_capacity':dynamic_load_capacity,
    'effective_megapixels':mega_pixels,
    'first_aid_person_count':person,
    'first_aid_piece_count':pieces,
    'fits_strapping_width':inch_fraction,
    'flow_rate_gpm':flow_rate_GPM,
    'frequency_count':number_no_seperator,
    'frequency_response_hz':frequency_rate_hz,
    'gauge':number_no_seperator,
    'g_force':g_force,
#     'handlebar_height':inch,
    'handle_length':inch_fraction,
    'height':inch_feet,
    'hole_diameter':inch_fraction,
    'horsepower':horsepower,
    'impression_length':inch_fraction,
    'impression_width':inch_fraction,
    'jaw_length':inch_fraction,
    'jaw_width':inch_fraction,
    'key_capacity':number_no_seperator,
    'key_count':number_no_seperator,
    'labels_per_minute':number_no_seperator,
    'length':inch_feet,
    'length_metric':mm,
    'lens_height':inch_fraction,
    'lens_width':inch_fraction,
    'lines_per_second':sec,
    'load_count':number_no_seperator,
    'lumens':number_with_sperator,
    'maximum_battery_life':hour,
    'maximum_bulb_life':hours,
    'maximum_burn_time':hour,
    'maximum_document_width':inch,
    'maximum_impressions':number_with_sperator,
    'maximum_opening':inch_mm,
    'maximum_recommended_age':year,
    'maximum_rpm':rpm,
    'maximum_temperature_reading':temperature,
    'maximum_thickness':mil,
    'maximum_torque':pound,
    'memory':gb_tb_mb,
    'milliamp_hours_m_ah':milliamp_hours,
    'maximum_recommended_age':year,
    'minimum_temperature_reading':temperature,
    'noise_reduction_rating_nrr':number_no_seperator,
    'offset_bond':pound,
    'page_count':page,
    'pages_per_minute':number_with_sperator,
    'page_yield':number_with_sperator,
    'pallet_quantity':number_with_sperator,
    'peak_transfer_speed':peak_transfer_speed,
    'piece_count':pieces,
    'plate_thickness':inch,
    'pod_capacity':number_no_seperator,
    'potency_mg':mg,
    'probe_length':inch,
    'processor_speed_g_hz':ghz,
    'projection':inch,
    'psi':psi,
    'rackable_load_capacity':pound,
    'random_access_memory_ram':ram,
    'range':feet_mi,
    'refresh_rate_hz':hz,
    'response_time_gt_g':ms,
    'results_timing':minute,
    'roll_length':feet_thousand_seperator,
    'scans_per_second':number_no_seperator,
    'screen_size_in':inch,
#     'seat_height':inch,
    'shackle_diameter':feet_mm,
    'shade_coverage':square_feet,
    'sheet_capacity':number_with_sperator,
    'sheet_count':number_with_sperator,
    'shelf_depth':inch,
    'shelf_life':year,
    'shelf_weight_capacity':pound,
    'static_load_capacity':pound,
    'steel_gauge':gauge,
    'storage_capacity':pound,
    'storage_capacity_compressed':gb_tb_mb,
    'storage_capacity_native':gb_tb_mb,
    'tab_count':number_no_seperator,
    'tabletop_thickness':inch,
    'teeth_per_inch':tpi,
    'tension':pound,
    'thickness':inch,
    'thickness_micron':micron,
    'thickness_mil':mil,
    'thickness_mm':mm,
    'trim_length':inch_fraction,
    'voltage':volt,
    'voltage_ac':volt,
    'voltage_dc':volt,
    'wattage':wat,
    'weight':pound,
    'weight_capacity':pound,
    'width':inch_feet,
#     'wire_gauge_awg':
}

In [103]:
# cb[cb['attribute'].astype(str)=='width']['value'].explode().value_counts()[500:]

In [104]:
def filter_empty_matches(dataframe, attribute, pattern):
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    empty_matches = matches.apply(lambda x: len(x) == 0)
    return dataframe[dataframe['attribute'].astype(str) == attribute][empty_matches]

def count_empty_matches(dataframe, attribute, pattern):
    print(f'Number of SKUs for {attribute}: '+str(len(dataframe[dataframe['attribute'].astype(str) == attribute])))
    matches = dataframe[dataframe['attribute'].astype(str) == attribute]['value'].apply(lambda x: re_extract(pattern,str(x)))
    return sum(len(match) == 0 for match in matches)


filtered_dataframes = {}
for attribute, pattern in patterns.items():
    filtered_dataframes[attribute] = filter_empty_matches(cb, attribute, pattern)
    
counts = {}
for attribute, pattern in patterns.items():
    counts[attribute] = count_empty_matches(cb, attribute, pattern)
#     print(f'Rows that need to be wiped for {attribute}: '+f': {counts[attribute]}')
   

    try:
        flattened_list = filtered_dataframes[attribute]['external_id'].to_list()
        if len(flattened_list) > 0:
            df_name = f'match_{attribute}'
            data = {'external_id': flattened_list, f'Q:{attribute}': '[]'}
            df_dict = {df_name: pd.DataFrame(data)}
            new_df = df_dict[df_name]
            print('Number of SKUs to be wiped: ' + str(len(new_df)))
            print('')
            print('')

            def get_df_name(df):
                name = [x for x in globals() if globals()[x] is df][0]
                return name

            def looks_good(customer, matches, attribute):
                today = pd.Timestamp.today().strftime("%Y-%m-%d")
                drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload'
                matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_{attribute}-{today}.csv', index=False)
            looks_good(client, new_df, attribute)
            
        else:
            print('No SKUs to be wiped for ' + attribute)
    except KeyError:
        print('Error: DataFrame with name ' + df_name + ' does not exist')

Number of SKUs for additional_range: 65
Number of SKUs to be wiped: 3


Number of SKUs for amperage: 266
Number of SKUs to be wiped: 1


Number of SKUs for appliance_cubic_feet: 84
No SKUs to be wiped for appliance_cubic_feet
Number of SKUs for arbor_hole: 7
No SKUs to be wiped for arbor_hole
Number of SKUs for bills_per_minute: 1
No SKUs to be wiped for bills_per_minute
Number of SKUs for bin_count: 214
No SKUs to be wiped for bin_count
Number of SKUs for blade_length: 974
No SKUs to be wiped for blade_length
Number of SKUs for blade_width: 579
No SKUs to be wiped for blade_width
Number of SKUs for bond_lbs: 1338
No SKUs to be wiped for bond_lbs
Number of SKUs for bond_time: 9
No SKUs to be wiped for bond_time
Number of SKUs for brew_capacity: 47
No SKUs to be wiped for brew_capacity
Number of SKUs for brightness_nit: 238
Number of SKUs to be wiped: 1


Number of SKUs for btu: 92
No SKUs to be wiped for btu
Number of SKUs for can_count: 30
No SKUs to be wiped for can_count
Number of S

# Curated - All attributes

In [ ]:
dateszs='2001-08-11'
dateszs='2001-11-01'
curation_col = f'Q:{attribut}'
params = {'customer_id': customer_id,
          'customer_name':customer_name,
         'dateszs':dateszs}

print('Start Query of Curated Data')
cb = query_from_file(file_name='query/curated_all_attributes_date_family.sql', params=params)
print(len(cb))

fill_dict = cb.groupby('external_id')['buckets'].last().to_dict()
cb['buckets'] = cb['buckets'].fillna(cb['external_id'].map(fill_dict))

In [4]:
color=cb[(cb['attribute'].astype(str)=='color')&(cb['buckets'].astype(str)=='Mailing & Packing Boxes')]
print(len(color))

3901


In [5]:
color['value'].explode().value_counts()

Brown                2510
White                 741
Kraft                 451
Tan                    35
Red                    25
Black                  22
["Pink","White"]       21
Blue                   21
Gold                   21
Yellow                 11
Green                  11
Silver                 11
Gray                    7
["Blue","Brown"]        6
["Gray","White"]        4
Clear                   2
["Black","White"]       1
["Black","Brown"]       1
Name: value, dtype: int64

In [6]:

color['Q:color']=color['value'].apply(lambda x: re.sub(r'Brown','Kraft',str(x)))
match_color=color[['external_id','Q:color']]


attribute='color'

In [7]:
match_color['Q:color'].explode().value_counts()

Kraft                2961
White                 741
Tan                    35
Red                    25
Black                  22
["Pink","White"]       21
Blue                   21
Gold                   21
Yellow                 11
Green                  11
Silver                 11
Gray                    7
["Blue","Kraft"]        6
["Gray","White"]        4
Clear                   2
["Black","Kraft"]       1
["Black","White"]       1
Name: Q:color, dtype: int64

In [23]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
def looks_good(customer, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/Ready For Upload' 
    matches.to_csv(f'{drive_path}/{client} --{get_df_name(matches)}match_cap-{today}.csv',index=False) 
looks_good(client, match_cap) 

# Uncurated

In [ ]:
formatted_attribute = 'sustainability'
buckets = """Office Chairs"""

# strategy_version_input=767
# attribute_id_input=2730

attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'


df = query_from_file(file_name='./query/custom_fields.sql', params=params)
print(len(df))
print(today)